In [3]:
#Importar librería gbq
import pandas as pd
import itertools
from google.cloud import bigquery



In [4]:
# Autenticación
client = bigquery.Client()

# crear archivo
archivo = '../../../Correos shows Mailing/Catalogo_mailing_grupos.xlsx'
excel_writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

lista_grupos = [
    'shows',
    'deportivos',
    'ficciones']

nombre_rdf = {
    "shows": "Shows", 
    "deportivos" : "Deportivos",
    "ficciones": "Ficciones"}


In [5]:
for show in range(len(lista_grupos)):

    query_grupos = """
    with consumo as (
    SELECT distinct dicc_shows._id, dicc_shows.title, usuarios_unicos.email
    FROM `conexion-datos-rdf.usuarios.usuarios_emisor` AS usuarios_unicos
    LEFT JOIN`conexion-datos-rdf.consumo_usuarios.consumo_mensual_content` AS consumo_usuarios
    ON usuarios_unicos.platform_id = consumo_usuarios.user_id
    LEFT JOIN `conexion-datos-rdf.dicc_medios.dicc_medios` AS dicc_medios
    ON consumo_usuarios.content_id = dicc_medios.media_id
    LEFT JOIN `conexion-datos-rdf.diccionarios.dicc_shows` AS dicc_shows
    ON dicc_medios.show_id = dicc_shows._id), 

    shows as ( 
    select distinct consumo.email 
    from consumo where consumo._id in (
        '5cc0d1173477b8633b077e41', '5f2ac786f815b70768cc3228', 
        '5e3825c34948a26cf6148327', '5cc330a7982853083c925a34',
        '603d4a8276a72f07eafc2e00', '5cc1cd433477b8633b077e4e', 
        '5c91d11241133e279ac3ae9c','60d23f724a86db086588f80e', 
        '5c91d11141133e279ac3ae9a','5ddc451635a6b907b3a311a2', 
        '5cc45fd8300fab43398c9501', '5e84f12d5efbf109f2113f7e', 
        '5ca2623760d338056118c923', '5f2b0f2674c51b07c018e151')),

    deportivos as ( 
    Select distinct consumo.email
    from consumo 
    left outer join shows
    on consumo.email = shows.email
    where consumo._id in (
        '633b35809eae4457a1c12a8b', '61f2dc0b55b34a084bb4040d', 
        '60c2d54896ecd50835f7023b','5f282d6ed9c4f53eefb9d3a6',
        '5eb2de321075c570328bf8ca','632244c62fe6940836b9b924',
        '5d02cb517fd8f34a99f13e0b','5cf98c6dc2b9a60768711e36')
        and shows.email is null),

    ficciones as ( 
    Select distinct consumo.email
    from consumo 
    left outer join shows
    on consumo.email = shows.email
    where consumo._id in (
        '632a06fdb7642562b6cb35a7','5f89b5c5f48bc20833b4ab2e',
        '60a2753c778b950838da9823','5e89fcdb6315583216a781f6',
        '5ffc2e0e9d178e63a402bfca','6165b91c61a232240994d193',
        '623b8332381bae08433d4381','5f15a254ba175108875c78c5',
        '5d110bba8d37311f4dd48459','611139e4c8b8c6084b6812c5',
        '5f80c9f70334d4082d84f385','60df3534541ab20860650422',
        '5f3f2ba9270a4f51cdd04109','5ddc451635a6b907b3a311a2',
        '5ddc4dc0487b8a07bd684d61','5ddc525cf4e0ae079d308bf9',
        '5ddc561b89d8670a2c41182c','5ddc5785487b8a07bd684d6e',
        '603d4a8276a72f07eafc2e00', '61e18e87124d6b390b1cc65d')
        and shows.email is null)

    select * from {0}

    """.format(lista_grupos[show])

    df_shows = client.query(query_grupos).result().to_dataframe( create_bqstorage_client=False)  
    df_shows["show"] = nombre_rdf[lista_grupos[show]]
    df_shows.to_excel(excel_writer, sheet_name=lista_grupos[show], index=False)

In [6]:
excel_writer.save()